<a href="https://colab.research.google.com/github/Sonic11-stack/face-recognizer/blob/main/Face_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
from zipfile import ZipFile
work_dir = os.getcwd()                                                  #Сохраняет текущий рабочий каталог.
with ZipFile(os.path.join(work_dir ,'/content/drive/MyDrive/Collab Notebooks/archive_1.zip'),'r') as zipobject:
#with ZipFile(os.path.join(work_dir ,'archive.zip'),'r') as zipobject:
  zipobject.extractall()

In [4]:
train_dir = os.path.join(work_dir,'train')       #Путь к каталогу train.
test_dir = os.path.join(work_dir,'test')         #Путь к тестовому каталогу.
os.makedirs(train_dir, exist_ok=True)  #Создание каталога для обучающих примеров.
os.makedirs(test_dir, exist_ok=True)   #Создание каталога для тестирования примеров.

In [5]:
train_ratio=0.8
test_ratio=0.2
dir_list=os.listdir(os.path.join(work_dir ,'105_classes_pins_dataset'))
print('The Number of Classes in the Dataset is:{}'.format(len(dir_list)))

The Number of Classes in the Dataset is:105


In [6]:
#Этот код разделит распакованный набор данных на обучающий и тестовый наборы в соответствии со значениями train_ratio и test_ratio.
from shutil import copyfile
import numpy as np

source_dir = os.path.join(work_dir ,'105_classes_pins_dataset')        #Распакованный каталог набора данных

dir_list = os.listdir(source_dir)

for folder in dir_list:
    data_dir = os.listdir(os.path.join(source_dir,folder))
    np.random.shuffle(data_dir)                            #Перетасовка примеров классов внутри набора данных.
    os.makedirs(os.path.join(train_dir , folder), exist_ok=True)
    os.makedirs(os.path.join(test_dir , folder), exist_ok=True)
    train_data = data_dir[:int(len(data_dir)*train_ratio+1)] #Разделение обучающего набора данных по отношению к train_ratio.
    test_data = data_dir[-int(len(data_dir)*test_ratio):]

    for image in train_data:
        copyfile(os.path.join(source_dir,folder,image) , os.path.join(train_dir,folder,image)) #Копирование обучающих файлов из набора данных в каталог обучения.

    for image in test_data:
       copyfile(os.path.join(source_dir,folder,image) , os.path.join(test_dir,folder,image))  #Копирование обучающих файлов из набора данных в каталог обучения.


In [7]:

print("The Number of Classes in the Training Set:{}".format(len(os.listdir(train_dir))))
print("The Number Of Classes in the Testing Set:{}".format(len(os.listdir(test_dir))))
print('For Verification of the Split.....')
print("The Number of Examples in the Class='pins_Katherine Langford' in the Dataset Directory:{}".format(len(os.listdir(os.path.join(source_dir,'pins_Katherine Langford')))))
print("The Number of Examples in the Class='pins_Katherine Langford' in the Train Directory:{}".format(len(os.listdir(os.path.join(train_dir,'pins_Katherine Langford')))))
print("The Number of Examples in the Class='pins_Katherine Langford' in the Test Directory:{}".format(len(os.listdir(os.path.join(test_dir,'pins_Katherine Langford')))))


The Number of Classes in the Training Set:105
The Number Of Classes in the Testing Set:105
For Verification of the Split.....
The Number of Examples in the Class='pins_Katherine Langford' in the Dataset Directory:226
The Number of Examples in the Class='pins_Katherine Langford' in the Train Directory:181
The Number of Examples in the Class='pins_Katherine Langford' in the Test Directory:45


In [8]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
rows=160  #Количество пикселей в строках для ввода.
cols=160  #Количество пикселей в столбцах для вывода.

In [9]:
trained_model=MobileNetV2(input_shape=(rows,cols,3),
                    include_top=False,
                    weights='imagenet')


trained_model.trainable=True  #Разморозка всех предварительно подготовленных слоев Mobile Net V2 для обучения.

trained_model.summary()

9406464/9406464 [==============================] - 1s 0us/step
Model: "mobilenetv2_1.00_160"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 160, 160, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 80, 80, 32)           864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 80, 80, 32)           128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReL

In [10]:
last_layer=trained_model.get_layer('out_relu')
last_layer_output=last_layer.output  #Сохраняет выходные данные последнего уровня Mobile Net V2.

In [11]:
#Функция обратного вызова, которая прекращает обучение, когда точность достигает 98%.
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy') > 0.98):
      print("\nReached 98% accuracy so cancelling training!")
      self.model.stop_training = True
callbacks=myCallback()

In [12]:
x = tf.keras.layers.GlobalAveragePooling2D()(last_layer_output)

x = tf.keras.layers.Dropout(0.8)(x)

x = tf.keras.layers.Dense(105,activation='softmax')(x)

model = tf.keras.Model(trained_model.input,x)

model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])
#Краткое описание модели.
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 160, 160, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 80, 80, 32)           864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 80, 80, 32)           128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 80, 80, 32)           0         ['bn_Conv1[0][0]']        

In [13]:
train_datagen = ImageDataGenerator(rescale=1/255,
                                 shear_range=0.2,
                                 zoom_range=0.2,
                                 horizontal_flip=True,
                                 rotation_range=40,
                                 width_shift_range=0.1,
                                 height_shift_range=0.1) # создание генератора обучающих данных (train_datagen) с использованием класса ImageDataGenerator

test_datagen = ImageDataGenerator(rescale=1/255)

train_generator=train_datagen.flow_from_directory(train_dir,
                                                  target_size=(rows,cols),
                                                  class_mode='categorical') #метод загружает изображения из указанной директории и применяет определенные трансформации

test_generator=test_datagen.flow_from_directory(test_dir,
                                                target_size=(rows,cols),
                                                class_mode='categorical') # Аналогично создается генератор тестовых данных (test_generator) для директории тестовых данных

Found 14084 images belonging to 105 classes.
Found 3463 images belonging to 105 classes.


In [22]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Определите Callback для сохранения модели
checkpoint_callback = ModelCheckpoint("model_weights.h5",
                                      monitor='val_loss',
                                      save_best_only=True,
                                      mode='min',
                                      verbose=1)

result=model.fit(train_generator,
                 validation_data=test_generator,
                 epochs=50,
                 callbacks=[checkpoint_callback],
                 verbose=1) #процесс обучения модели с использованием метода fit


Epoch 1/50
441/441 [==============================] - ETA: 0s - loss: 4.9786 - accuracy: 0.0143
Epoch 1: val_loss improved from inf to 4.69509, saving model to model_weights.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


441/441 [==============================] - 157s 285ms/step - loss: 4.9786 - accuracy: 0.0143 - val_loss: 4.6951 - val_accuracy: 0.0289
Epoch 2/50
441/441 [==============================] - ETA: 0s - loss: 4.5352 - accuracy: 0.0292
Epoch 2: val_loss improved from 4.69509 to 4.21452, saving model to model_weights.h5
441/441 [==============================] - 131s 296ms/step - loss: 4.5352 - accuracy: 0.0292 - val_loss: 4.2145 - val_accuracy: 0.0537
Epoch 3/50
441/441 [==============================] - ETA: 0s - loss: 4.2204 - accuracy: 0.0574
Epoch 3: val_loss improved from 4.21452 to 3.81120, saving model to model_weights.h5
441/441 [==============================] - 135s 306ms/step - loss: 4.2204 - accuracy: 0.0574 - val_loss: 3.8112 - val_accuracy: 0.1022
Epoch 4/50
441/441 [==============================] - ETA: 0s - loss: 3.8254 - accuracy: 0.1054
Epoch 4: val_loss improved from 3.81120 to 3.43640, saving model to model_weights.h5
441/441 [==============================] - 127s 289m

In [16]:
from tensorflow.keras.models import load_model  # Добавлен импорт

model_dict = load_model('model_weights.h5')

path="/content/drive/MyDrive/Collab Notebooks/32.jpg"  #Путь к целевому изображению, подлежащему прогнозированию.

classes=train_generator.class_indices
class_names=list(classes.keys())   #Список имен классов.
img = tf.keras.preprocessing.image.load_img(
    path, target_size=(rows, cols)
)

img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)
img_array=img_array/255.

score = model_dict.predict(img_array)

print(
    "The Person in the Image is {} ."
    .format(class_names[np.argmax(score)][5:].title())
)

1/1 [==============================] - 2s 2s/step
The Person in the Image is Chris Pratt .
